<a href="https://colab.research.google.com/github/RDMotta/ExampleGemini/blob/main/Google_IA_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exemplo de uso da API para consultas usando embedings

Instalando no ambiente a biblioteca

In [1]:
!pip install -U -q google-generativeai

Importando configurações iniciais.

In [48]:
import numpy as np
from numpy import dot
import pandas as pd
import google.generativeai as genai

Import APIKey from colab

In [5]:
from google.colab import userdata
GOOGLE_APY_KEY=userdata.get('APIKey')
genai.configure(api_key=GOOGLE_APY_KEY)

Listagem dos modelos disponiveis no Gemini para Embeding

In [8]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


Listagem de modelos disponiveis no Gemini para Content

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

Exemplo de geração de embedings

documentação> https://ai.google.dev/gemini-api/tutorials/document_search

In [33]:
model_embedding = "models/embedding-001"

In [17]:
title="Como treinar seu cachorro"
sample_text=("Titulo: Como treinar seu cachorro"
"\n"
"Artigo completo:\n"
"\n"
"Primeiro tenha um cachorro, segundo de ordem de maneira que seja firme")

embedding = genai.embed_content(
    model=model_embedding,
    content=sample_text,
    title=title,
    task_type="RETRIEVAL_DOCUMENT"
)

print(embedding)


{'embedding': [-0.0009524487, -0.042919304, -0.041207235, 0.0032501654, -0.009115104, 0.018858176, -0.0030579267, 0.0017425981, 0.0018337128, 0.049200147, 0.0029161791, -0.0031915088, 0.011812618, -0.042146023, 0.011349151, 0.0381694, 0.0004834972, 0.029862138, -0.0100270035, -0.010527324, 0.0046214834, 0.007309757, 0.012035757, 0.025599897, -0.0055348063, -0.01751613, 0.036848374, -0.02614904, -0.016761616, 0.021135509, -0.06466269, 0.047979027, -0.04014315, 0.0036526993, 0.014840813, -0.0708564, -0.043158446, 0.014586403, 0.004318746, -0.0011126816, -0.032118194, -0.027418938, -0.05291055, -0.025463503, 0.03720554, 0.015304084, -0.030704983, 0.024597727, 0.04243365, -0.063901655, 0.0102254925, -0.030952081, 0.09755379, -0.006206039, -0.014916176, -0.051302373, 0.07342658, 0.014496983, -0.017368183, -0.0061194818, -0.022130173, 0.012262652, 0.020112162, -0.0017378242, 0.01723077, -0.014321777, -0.069209516, 0.021813823, 0.026047511, 0.0020400211, 0.015986327, -0.027201448, -0.02666939

Listando documentos para serem usados em buscas

In [40]:
DOCUMENT_1 = {
    "Titulo":"Encerramento de Contratos",
    "Conteudo": "Agora, os administradores podem cadastrar novos motivos de cancelamento e habilitá-los para unidades de negócios específicas. Na mesma tela, também será possível ativar/desativar o fluxo de distrato por e-mail para cada motivo de cancelamento, permitindo também a seleção dos destinatários das notificações de distrato."
}

DOCUMENT_2 = {
    "Titulo":"Comunicação de Distratos",
    "Conteudo": "Quando ocorrer uma solicitação de distrato, o sistema notifica o cliente com base no(s) motivo(s) selecionado(s), enviando o e-mail na língua oficial do país do cliente. Não sendo Português ou Espanhol, o padrão será Inglês."
}

DOCUMENT_3 = {
    "Titulo":"Agrupamento de regras",
    "Conteudo": "Visando a facilidade de configurar contratos com cálculos que muitas vezes se repetem, foi disponibilizado um agrupamento de regras no sistema, assim ao configurar um agrupamento para as regras na unidade especifica, o sistema possibilita carregar e configurar no contrato as regras do agrupamento"
    "\n"
    "Após definir um agrupamento em um item de contrato, também é possivel filtrar os mesmos pela tela de contratos, facilitando a localização de contratos com os tipos de configurações especificas"
}

documents = [DOCUMENT_1, DOCUMENT_2, DOCUMENT_3]

Usando Pandas para criar DataFrame e poder trabalhar os dados

In [41]:
df = pd.DataFrame(documents)
df

,Titulo,Conteudo
0,Encerramento de Contratos,"Agora, os administradores podem cadastrar novo..."
1,Comunicação de Distratos,"Quando ocorrer uma solicitação de distrato, o ..."
2,Agrupamento de regras,Visando a facilidade de configurar contratos c...


Criando função para gerar novos embedins com base nos dados de entrada.

In [42]:
def generate_embedings(title, text):
  embeddings = genai.embed_content(model=model_embedding,
                                  content=text,
                                  title=title,
                                  task_type="RETRIEVAL_DOCUMENT")
  return embeddings["embedding"]

Atualizando DataFrame com nova coluna para vincular o embedding gerado

In [43]:
df["Embeddings"] = df.apply(lambda row: generate_embedings(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Encerramento de Contratos,"Agora, os administradores podem cadastrar novo...","[0.005925163, -0.022211492, -0.08218023, 0.006..."
1,Comunicação de Distratos,"Quando ocorrer uma solicitação de distrato, o ...","[-0.0009933693, -0.040826205, -0.0450991, 0.00..."
2,Agrupamento de regras,Visando a facilidade de configurar contratos c...,"[0.03992304, -0.0023297227, -0.016508946, 0.04..."


Gerando consultas em cima dos dados da base

In [56]:
def consultar_dados(query, base, model):
  embeddings = genai.embed_content(model=model_embedding,
                                  content=query,
                                  task_type="RETRIEVAL_QUERY")
  produtos_escalares =  np.dot(np.stack(base["Embeddings"]), embeddings["embedding"])
  #localizar o argumento maximo (mais proximo da informação pesquisada)
  indice = np.argmax(produtos_escalares)
  #localiza no DataFrame o conteúdo com o mesmo indice
  return base.iloc[indice]["Conteudo"]

Usando a função para pesquisar informações dentro da base de dados que foi "Embedada"

In [57]:
consulta = "Como fazer café?"
resultado = consultar_dados(consulta, df, model_embedding)
print(resultado)

Visando a facilidade de configurar contratos com cálculos que muitas vezes se repetem, foi disponibilizado um agrupamento de regras no sistema, assim ao configurar um agrupamento para as regras na unidade especifica, o sistema possibilita carregar e configurar no contrato as regras do agrupamento
Após definir um agrupamento em um item de contrato, também é possivel filtrar os mesmos pela tela de contratos, facilitando a localização de contratos com os tipos de configurações especificas
